**1a)**
Implement a function that computes the binary entropy

The function takes as input one feature vector x and the corresponding label y. Binary entropy means binary labels y=0/1

**1b)**
Implement a function that takes one given feature of a dataset, and finds the best split (the split that
minimises the entropy) for the data. It is common to iterate over the data feature values, and calculate
the entropy for the subsets that are larger or smaller (or equal) than the current value in the iteration.
return the split that minimises the entropy and the corresponding entropy for the split

**1c)**
Implement a function that takes a dataset (nsamples × nfeatures) and finds the single best split (the split
with the least entropy) across all features. That is the function find_best_split. Return the best split
found.

**1d)**
Using the functions you implemented above, write your own version of the Classification Tree algorithm,
using recursion. Remember to include a parameter specifying the maximum depth of the tree to prevent
overfitting
![Ethem univariate tree construction](img/decision_tree.png)

In [23]:
MINIMUM_ENTROPY = 0.1

import numpy as np
from numpy import log2
import pandas as pd
import matplotlib.pyplot as plt
import sys

def entropy(y: np.array):
    counts = np.bincount(y)
    p_0 = counts[0] / len(y)
    p_1 = 1 - p_0
    log2p0 = 0 if p_0 == 0 else log2(p_0)
    log2p1 = 0 if p_1 == 0 else log2(p_1)
    return -p_0 * log2p0 - p_1 * log2p1


def split_feature(x: np.array, y: np.array):
    min_impurity_index = None
    min_impurity = sys.float_info.max
    # Iterate over all values in the features in x.
    i_sorted = np.argsort(x)
    x = x[i_sorted]  
    y = y[i_sorted]
        
    for i in range(len(x)-1):
        sub_1 = y[:i+1]
        sub_2 = y[i+1:]
        e_1 = entropy(sub_1)
        e_2 = entropy(sub_2)
        impurity = e_1 * len(sub_1) / len(x) + e_2 * len(sub_2) / len(x)
        if impurity < min_impurity:
            min_impurity = impurity
            min_impurity_index = i

    return x[min_impurity_index], min_impurity

def find_best_split(X: np.array, y: np.array):
    M = X.shape[1]
    min_impurity_global = sys.float_info.max
    min_impurity_feature_index = None
    split_value = None
    for i in range(M):
        feature = X[:, i]
        value, min_impurity = split_feature(feature, y)
        if min_impurity < min_impurity_global:
            min_impurity_global = min_impurity
            min_impurity_feature_index = i
            split_value = value
            # There might be a break here if we got a entropy near 0.
            if min_impurity_global < 1e-20:
                break

    return min_impurity_feature_index, split_value


class Tree:
    # Initialize variables
    def __init__(self, level, left):
        self.__split_value = None
        self.__feature_index = None
        self.__left_child = None
        self.__right_child = None
        self.__value = None
        self.__level = level + 1
        if left is None:
            self.__direction = "Root"
        else:
            self.__direction = "Left" if left else "Right"

    # Implements the "GenerateTree"-function from Fig. 9.3 in the book.
    def fit(self, data, labels):

        if entropy(labels) < MINIMUM_ENTROPY:  # Stopping condition for recursion
            # Pick the label that has the majority. To be used if we are a leaf-node
            self.__value = 1 if np.mean(labels) > 0.5 else 0
            print(self.__direction,"leaf node at level",self.__level, "with value", self.__value)
            return

        feature_index, split_value = find_best_split(data, labels)

        self.__feature_index = feature_index
        self.__split_value = split_value
        print(self.__direction,"node at level", self.__level, ". Feature index:", feature_index, ", split value",split_value)

        feature = data[:, feature_index]
        indexes_left = np.where(feature <= split_value)
        indexes_right = np.where(feature > split_value)

        # Create branches
        self.__left_child = Tree(self.__level, True)
        self.__right_child = Tree(self.__level, False)

        # Generate sub-trees.
        
        indexes_left_ = indexes_left[0]
        left_ = data[indexes_left_, :]
        if len(left_) > 0 :
            self.__left_child.fit(left_, labels[indexes_left])
        else : 
            self.__left_child = None
        indexes_right_ = indexes_right[0]
        right_ = data[indexes_right_, :]
        if len(right_) > 0:
            self.__right_child.fit(right_, labels[indexes_right])
        else:
            self.__right_child = None

    # Find leaf corresponding to row
    def predict(self, x):
        if self.__left_child is None and self.__right_child is None:  # We are a leaf-node
            return self.__value

        feature_val = x[self.__feature_index]
        if feature_val <= self.__split_value:
            return self.__left_child.predict(x)
        else:
            return self.__right_child.predict(x)


data = pd.read_csv("data/blobs.csv", skiprows=1, delimiter=" ")
# y = data[0,:].to_numpy(dtype=int)
y = data.iloc[:, 0].to_numpy()
X = data.iloc[:, 1:].to_numpy()

# y = 
# X = 

decision_tree = Tree(0,None)
decision_tree.fit(X, y)
print("Voila!")


Root node at level 1 . Feature index: 0 , split value 0.5835
Left node at level 2 . Feature index: 1 , split value 0.4348
Left leaf node at level 3 with value 0
Right node at level 3 . Feature index: 1 , split value 0.7322
Left node at level 4 . Feature index: 0 , split value 0.4281
Left leaf node at level 5 with value 0
Right leaf node at level 5 with value 1
Right leaf node at level 4 with value 1
Right leaf node at level 2 with value 1
[-0.2096 -0.0599]
0
Voila!


**1e)**
Test your implementation on the datasets in blobs.csv and flame.csv. Plot the data, and the regions
found by the tree.


In [23]:
test = X[153, :]
print(test)
print(decision_tree.predict(test))


**1f)** Vizualise tree 
https://levelup.gitconnected.com/binary-tree-implementation-and-visualization-in-python-2f4782887ca2